Vorbereitung der Umgebung
Benötigte Packete
-nympy
-matplotlib
-gstools
-lhsmdu

-jupyter-notebook Umgebung (z.B. jupyterlab)

- jupyterlab mit "jupyter-lab" in Konsole ausführen



In [ ]:
#Importiere benötigte Pakete

import numpy as np
from matplotlib import pyplot as plt
import gstools as gs
import lhsmdu as lhs 
import pandas as pd

# wenn's knallt, müsst ihr Bibliotheken nachinstallieren


In [ ]:
#Importiere alle für Hand-Ons benötigten Funktionen
from ml_pareto_kriging import calculateKrigingModel, plotKrigModel, plotCovarianceModel, getSEmapOfKrigModel, getBestCovarianceModel, plotFunction2D
from ml_pareto_kriging import getPointWithMaxValue, addOnePoint, getMSE, analyseFunction 
from ml_pareto_kriging import addRandomPoints, addEdgePoints, getRandomPoints, getGridPoints, getCenteredGridPoints, getCenteredRandomGridPoints, getCenteredRandomGridCombinedPoints, getCenteredRandomGridCombinedPointsWithEdges, getInitPoints, getLHSPoints, getMCPoints
from ml_pareto_kriging import f1, f2, f3, f_herbie_1, f_herbie_2, f_herbie_3, f_herbie_4, f_herbie_complex, f_rosenbrock, f_zum_versagen_von_kriging, f_shubert_1_not_for_hand_ons, f_shubert_2_not_for_hand_ons, f_parabolid, f_hyperbolic_parabolid, f_eggholder, f_six_hump_camel, f_three_hump_camel, f_branin
from ml_pareto_kriging import calculateCovarianceModel, analyseMSEvsN

# wenn's knallt, müsst ihr Bibliotheken nachinstallieren

#####################################################
##############  HANDS-ON: Einleitung   ##############
#####################################################

# Erklärung der vorhanden Funktionen (zum Nachschlagen bei Fragen)

distribution_type
"RandomPoints" (bzw. "else-case),           #zufällige verteilung
"GridPoints"                                #im gitter angeordnet mit randpunkten
"CenteredGridPoints"                        #im gitter angeordnet, ohne randpunkte
"CenteredRandomGridPoints"                  #in gitter aufgeteilt und im quadranten eine zufallszahl gewählt
"CenteredRandomGridCombinedPoints"          #kombination von random-grid und random punkten
"CenteredRandomGridCombinedPointsWithEdges" #kombination auf random-grid und random punkten +4 eckpunkte
"LHS"                                       #latin hypercube sampling (LHS)
"MC"                                        #monte carlo
x_p,y_p = getInitPoints(distribution_type, n_datapoints, min_x,max_x, min_y,max_y)

In dieser Funktion werden die einzelnen Funktionen aufgerufen: 
- addRandomPoints
- addEdgePoints
- getRandomPoints
- getGridPoints
- getCenteredGridPoints
- getCenteredRandomGridPoints
- getCenteredRandomGridCombinedPoints
- getCenteredRandomGridCombinedPointsWithEdges
- getLHSPoints
- getMCPoints
- - - - - - - - -     
plotFunction2D(f, ax) #plotted die in handons verwendete funktionen
- - - - - - - - -   
kriging_type
"simple"
"ordinary"
"universal_linear"
"universal_quadratic"
calculateKrigingModel(kriging_type, model_in, x_in, y_in, z_in) 
- - - - - - - - -   
plotKrigModel #plotet das kriging-model
- - - - - - - - -   
plotCovarianceModel #plotet das kovarianzmodel
- - - - - - - - -   
getSEmapOfKrigModel #berechnet quadratische abweichung von wahrer funktion
- - - - - - - - -   

        "Gaussian": gs.Gaussian,
        "Exponential": gs.Exponential,
        "Matern": gs.Matern,
        "Stable": gs.Stable,
        "Rational": gs.Rational,
        "Circular": gs.Circular,
        "Spherical": gs.Spherical,
        "SuperSpherical": gs.SuperSpherical
getBestCovarianceModel(x_p, y_p, z_p, textoutput = False, models) #fittet die übergebenen Kovarianzmodelle und gibt das model mit dem besten pseudo-r2 wert zurück
- - - - - - - - -   
getPointWithMaxValue #gibt den punkt x,y mit dem größten erwarteten Krigingmodelfehler zurück
- - - - - - - - -   
addOnePoint #fügt einen punkt zum bestehen array
- - - - - - - - -   
getMSE #berechnet mean squared error von einem array
- - - - - - - - - - - -
Mit dieser Funktion kann der MSE in Abhängigkeit von der Anzahl der Startpunkte untersuchen werden.
Man kann in diesem notebook definierte Funktion damit untersuchen
Die Logik führt zum Einen den Fit mit allen Punkten durch, zum Anderen wird zusätzlich mit Aufsplittung in Startpunkte + größter SE das Model laufend verbessert
analyseFunction(f1, n_points_to_fit, n_runs, init_fraction, fig, ax1, ax2, ax3, ax4)
- - - - - - - - - - - -
f1, f2, f3, .... sind die definierten Funktionen als Klassen, die in Handons verwendet werden, mit
 .getValue(x,y) zur Rückgab des funktionwertes
 .getRange() zur Rückgabe des Wertebereichs

Liste der Funktionen:
- f1
- f2
- f3
- f_herbie_1
- f_herbie_2
- f_herbie_3
- f_herbie_4
- f_rosenbrock
- f_zum_versagen_von_kriging
- f_parabolid
- f_hyperbolic_parabolid
- f_eggholder
- f_six_hump_camel
- f_three_hump_camel
- f_branin 

Funktionsklasse
#Erstelle deine eigne funktion, oder verwende eine der bereits definierten
    class f_deine_eigene_funktion:
        def __init__(self):
            #definiere Wertebereich der Funktion
            self.min_x = -1
            self.max_x = 1
            self.min_y = -1
            self.max_y = 1

        def getRange(self):
            return self.min_x, self.max_x, self.min_y, self.max_y

        def getValue(self, x, y):
            #definiere funktion
            #z.b. return x + y (geneigte platte)
            #z.b. return x**2 + y**2 Paraboloid
            return x**2-y**2
            
Weitere Funktionen: https://www.sfu.ca/~ssurjano/optimization.html

# ###########################################################################################
# HANDS-ON: Bestimme eine Fitfunktion die am besten eine wahre Funktion beschreibt ######
# Optimierungsziel: Minimiere Anzahl der Stützpunkte, Minimiere MSE ############################
# ###########################################################################################

In [ ]:
#wählt die funktion aus und schaut diese an
f_current = f_eggholder()

#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_current.getRange()

fig, ax = plt.subplots()
plotFunction2D(f_current, ax)

# Stützpunkte: Punkte, an denen die wahre Funktion ausgewertet werden muss

In [ ]:
# Falls ihr verstehen wollt, wie die Startpunkte verteilt werden, wählt entsprechende Funktion aus

fig, ax = plt.subplots()
ax.title.set_text('Auswahl Stützpunkte')

#definiere parameter
n_datapoints = 64
#distribution_type
# / "MC"
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
# / "LHS"
x_p,y_p = getInitPoints("LHS", n_datapoints, min_x,max_x, min_y,max_y)


plt.xlim(min_x, max_x)
plt.ylim(min_y, max_y)
ax.scatter(x_p,y_p, color = "red")





# Beispiel für einen Parameter-Satz (werden später große Anzahl unterschiedlicher Parametersätze untersuchen)

In [ ]:
##############################################################
###############   run        #################################
##############################################################

#wähle die funktion aus (siehe Liste der Funktionen von oben) -> optional
f_current = f_eggholder()   

#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_current.getRange()

n_start_points = 25 # <- Andere Anzahl der Startpunkte ausprobieren
x_p,y_p = getInitPoints("LHS", n_start_points, min_x,max_x, min_y,max_y) #<- Andere Verteilugnsfunktion der Startpunkte ausprobieren

#berechne die funktionswerte für startpunkte
z_p = f_current.getValue(x_p, y_p)

#fitte variogram manuell (so übernehmen)
best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

#berechne kriging-model
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p) #<- anderes KrigingModel ausprobieren (optional)

#man kann das feld, sqrt(varianz) = std.abw 
fig, (ax1, ax2) = plt.subplots(1,2)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
fig.set_size_inches(15,5)

######## Ergebnis #################

# Ihr seht links die vorhergesagte Funktion mit roten Stützpunkten, rechts dazu gehöriger Fehler (MSE). Die wahre Funktion habt ihr im ersten Abschnitt geplottet
# Ziel ist, den mittleren quadrierten Fehler MSE (im rechten Bild) zu minimieren, dann ist die Vorhersage der Funktion am "optimalsten"

# Aufgabe zur Pareto-Front: Vorhersagegenaugkeit vs. Datenpunkteanzahl

Euch steht die Funktion analyseMSEvsN() zur Verfügung,
Mit dieser Funktion kann der MSE in Abhängigkeit von der Anzahl der Startpunkte untersucht werden. Damit werden MSE vs Datenpunktenanzahl aufgetragen und so kann die Pareto-Front für die optimalen Parameter bestimmt werden.
Man kann in diesem notebook definierte Funktion damit untersuchen oder eine einfachere (siehe oben) untersuchen


In [ ]:
##############################################################
######## Parameter frei wählbar ##############################
##############################################################

#wähle die Funktion zur statistischen Analyse aus
f_funktion_zum_analysieren = f_eggholder() # f_eggholder ist die komplexeste Funktion, alle anderen sind einfacher

n_points_to_fit_min = 10 #minimale Anzahl der stützpunkte (nicht kleiner als 10 wählen, Berechnung könnte abstürzen)
n_points_to_fit_max = 50 #maximale Anzahl der stützpunkte (nicht größer als 100 wählen)



start_points_distribution = ["LHS",'CenteredRandomGridCombinedPointsWithEdges', 'MC'] # Mögliche Werte: ("MC", "RandomPoints" , "GridPoints", "CenteredGridPoints","CenteredRandomGridPoints", "CenteredRandomGridCombinedPoints","CenteredRandomGridCombinedPointsWithEdges", "LHS")
covariance_function = ["SelectBest"] # Mögliche Werte:  ("SelectBest", "Gaussian","Exponential","Matern","Stable","Rational","Circular","Spherical","SuperSpherical") :Note "SelectBest" wählt aus allen functionen die beste, wodurch die Berechnungszeit um Faktor ~10 steigt
krig_type = ["universal_quadratic"] # Mögliche Werte: ("simple","ordinary","universal_linear","universal_quadratic")

n_runs_per_parameter_set = 20 #anzahl der Wiederholungen mit gleichen Parametern (übertreibt nicht, max 50) -> es wird Mittelwert aller Runs berechnet
n_points_to_analyse = 200 #Max. Anzahl der verschiedenen Parameter-Sets, die getestet werden, bzw. alle Parameterpermutationen 

######## run (1-10min) je nach gewählte Parameterbereiche #####################################

analysed_values = analyseMSEvsN(f_funktion_zum_analysieren,n_points_to_fit_min, n_points_to_fit_max, start_points_distribution, covariance_function, krig_type, n_runs_per_parameter_set, n_points_to_analyse)


# plotting

# merkt euch die Farben ;) habe nicht geschafft es in der Legende auszugeben
color_dict = { 'LHS':'red', 'CenteredRandomGridCombinedPointsWithEdges':'blue', 'RandomPoints':'black', 'GridPoints':'green','MC':'purple','CenteredGridPoints':'grey','CenteredRandomGridPoints':'yellow','CenteredRandomGridCombinedPoints':'brown' }


column_names = ['Kriging', 'Covariance', 'Distribution', 'nPoints', 'MSE']
df = pd.DataFrame(analysed_values, columns=column_names)
fig, (ax1) = plt.subplots(1,1)
plt.scatter(df['nPoints'], df['MSE'], color=[ color_dict[i] for i in df['Distribution']], s = 200)
plt.yscale('log') # <- macht Plot übersichtlicher
plt.xlabel('nPoints')
plt.ylabel('MSE')
plt.title('MSE vs nPoints')
fig.set_size_inches(20,15)

#p.s. habe nicht geschafft die entsprechenden Werte der Farben in Legende zu plotten, gerne anpassen

# Falls ihr die Werte in Excel anschauen wollt, hier auskommentieren und output in Excel copy-pasten
#for sublist in analysed_values:
#    print(";".join(map(str, sublist)))

# Aufgaben:
- Welche der dargestellten Punkte bilden die Pareto-Front ?
- Könnt ihr ableiten, mit welchen Parametern man die optimalste Genauigkeit (je nach Startpunktenanzahl) erzielt ?
- (Optional) Andere Funktion analysieren bzw. andere Parameter (z.B. anderes Kriging Model) <- ist nicht der Schwerpunkt der Session, nur falls ihr euch langweilt